# Connect to Jötunn cluster

In [92]:
import ipyparallel as ipp
# Need to have sshserver as we otherwise get a timeout error
cli = ipp.Client(profile="jotunn", sshserver='sigurdur14@jotunn.rhi.hi.is')


In [93]:
# We can see all engines
# We can get each engine
cli[:]

<DirectView [0, 1]>

In [94]:
# We can call just one engine and display the output
dr1 = cli[0].execute("print(2+3)")
dr1

<AsyncResult(execute): pending>

In [95]:
dr1.display_outputs()

5


In [96]:
# Client gets the direct_view()
dv = cli.direct_view()
dv

<DirectView all>

##  Run some commands in parallel

In [97]:
sk1 = dv.execute("print(2+2)")
sk1

<AsyncResult(execute): pending>

In [98]:
sk1.display_outputs()

[stdout:0] 
4
[stdout:1] 
4


## MPI: Get process rank and number of total processes

In [99]:
%%px
from mpi4py import MPI
comm = MPI.COMM_WORLD
print(f"Hello World from rank {comm.Get_rank()}. total ranks={comm.Get_size()}")

[stdout:0] Hello World from rank 0. total ranks=2


[stdout:1] Hello World from rank 1. total ranks=2


## Progress bars for running these MPI commands.
Times each engine and the total

In [100]:
%%px --progress-after 0
from mpi4py import MPI
comm = MPI.COMM_WORLD

import random
import time
from tqdm.notebook import tqdm

print(MPI.Get_processor_name(), flush=True)
scale  = 0.1 * (1+comm.Get_rank())
for i in tqdm(range(25)):
    time.sleep(scale * random.random())


%px:   0%|          | 0/2 [00:00<?, ?tasks/s]

[stdout:0] jotunn.rhi.hi.is


[stdout:1] jotunn.rhi.hi.is


%px:   0%|          | 0/2 [00:00<?, ?tasks/s]

[output:0]

  0%|          | 0/25 [00:00<?, ?it/s]

[output:1]

  0%|          | 0/25 [00:00<?, ?it/s]

%px: 100%|██████████| 2/2 [00:02<00:00,  1.24s/tasks]


In [101]:
%px a=5

In [102]:
%px print(a)

[stdout:0] 5


[stdout:1] 5


In [103]:
r = cli.direct_view()

In [104]:
r.execute("print(2)")

<AsyncResult(execute): pending>

In [105]:
_.display_outputs()

[stdout:0] 
2
[stdout:1] 
2


In [106]:
%px import numpy as np; kka = np.array([1,2,3,4,5])

In [107]:
%px print(kka)

[stdout:0] [1 2 3 4 5]


[stdout:1] [1 2 3 4 5]


In [108]:
%%px
import os
print(os.getcwd())
print(os.listdir(os.getcwd()))

[stdout:0] /home/sigurdur14
['.bash_logout', '.bash_profile', '.bashrc', '.ssh', '.bash_history', '.emacs.d', 'pr.cpp', 'nohup.out', 'mylog.out', 'a.out', 'lab2', '.python_history', '.cache', '.local', '.lesshst', 'launch-python-for-ipy-ssh~', '.ipython', 'launch-python-for-ipy-ssh', '=1.0.1', 'jotunn_pip3_requirements.txt']


[stdout:1] /home/sigurdur14
['.bash_logout', '.bash_profile', '.bashrc', '.ssh', '.bash_history', '.emacs.d', 'pr.cpp', 'nohup.out', 'mylog.out', 'a.out', 'lab2', '.python_history', '.cache', '.local', '.lesshst', 'launch-python-for-ipy-ssh~', '.ipython', 'launch-python-for-ipy-ssh', '=1.0.1', 'jotunn_pip3_requirements.txt']


In [109]:
%%px

def mpi_example():
    from mpi4py import MPI
    comm = MPI.COMM_WORLD
    return f"Hello World from rank{comm.Get_rank()}. total ranks={comm.Get_size()}"


In [110]:
bc = cli.broadcast_view()

In [111]:
test = bc.execute("print(2)")

In [112]:
test.display_outputs()

[stdout:0] 
2
[stdout:1] 
2


## MPI: only targeting one engine at a time

In [113]:
%%px --targets 0
from mpi4py import MPI
comm = MPI.COMM_WORLD
print(f"Hello World from rank {comm.Get_rank()}. total ranks={comm.Get_size()}")

[stdout:0] Hello World from rank 0. total ranks=2


In [114]:
%%px --targets 1
from mpi4py import MPI
comm = MPI.COMM_WORLD
print(f"Hello World from rank {comm.Get_rank()}. total ranks={comm.Get_size()}")

[stdout:1] Hello World from rank 1. total ranks=2


## Direct_view() get hostname
We only have ssh login access to one machine on jotunn to start engines

so it should be host jotunn.rhi.hi.is

In [115]:
import socket
engine_hosts = cli.direct_view().apply_async(socket.gethostname).get_dict()
engine_hosts

{0: 'jotunn.rhi.hi.is', 1: 'jotunn.rhi.hi.is'}

In [117]:
host_engines = {}

for eid, host in engine_hosts.items():
    if host not in host_engines:
        host_engines[host] = []
    host_engines[host].append(eid)

host_engines

{'jotunn.rhi.hi.is': [0, 1]}